# Find all flights from one airport to another from historical traffic data

#### Between two European airports, with max one connection, with the same company and an acceptable connecting time

In [1]:
import pandas as pd
from datetime import datetime, timedelta

#### <br> Example with a trip from Toulouse, France (LFBO) to Vienna, Austria (LOWW) mid-June

In [2]:
from_airport = 'LFBO'
to_airport = 'LOWW'
min_connecting_time = 1 # in hour
max_connecting_time = 2 # in hour

#### <br> Load airports and airlines table from openflights.org

In [3]:
airports_url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
airlines_url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat'

airports_df = pd.read_csv(airports_url, header=None, names=['id', 'name', 'city', 'country', 'iata',
                                                            'icao', 'lat', 'lon', 'alt', 'timezone',
                                                            'dst', 'tz', 'type', 'source'])
airlines_df = pd.read_csv(airlines_url, header=None, names=['id', 'name', 'alias', 'iata', 'icao',
                                                           'callsign', 'country', 'active'])

In [4]:
def get_city(airport_icao):
    ans = airport_icao
    if not airports_df.query('icao == @airport_icao').empty:
        df = airports_df.query('icao == @airport_icao')
        ans = df.city.item() + "-" + df.name.item()
    return ans

In [5]:
def get_airline(airline_icao):
    ans = airline_icao
    if not airlines_df.query('icao == @airline_icao').empty:
        ans = airlines_df.query('icao == @airline_icao').name.item()
    return ans

#### <br> Load the so6 reference traffic file (e.g. one year before) as a pandas dataframe

In [6]:
my_so6_traffic_file = '20180617_m3.so6'
columns = ['segment_identifier', 'flight_origin', 'flight_destination', 'aircraft_type', 'time_begin',
           'time_end', 'fl_begin', 'fl_end', 'status', 'callsign', 'date_begin', 'date_end', 'lat_begin',
           'lon_begin', 'lat_end', 'lon_end', 'flight_identifier', 'sequence', 'length', 'parity']

parser = lambda x,y: pd.datetime.strptime(x + y, '%y%m%d%H%M%S')

df = pd.read_csv(my_so6_traffic_file, sep=' ', header=None, names=columns, 
                 parse_dates={'datetime_begin': ['date_begin','time_begin'], 
                              'datetime_end': ['date_end','time_end']}, date_parser=parser)

coord_cols = ['lat_begin','lon_begin','lat_end','lon_end']
df[coord_cols] = df[coord_cols].apply(lambda x: x / 60)

traffic_df = df[['callsign','datetime_begin','datetime_end','flight_origin','flight_destination',
                'fl_begin', 'fl_end', 'lat_begin', 'lat_end', 'lon_begin', 'lon_end', 
                 'aircraft_type','length','flight_identifier']]
traffic_df = traffic_df.query('length > 0') # filter null segments
traffic_df.head()

,callsign,datetime_begin,datetime_end,flight_origin,flight_destination,fl_begin,fl_end,lat_begin,lat_end,lon_begin,lon_end,aircraft_type,length,flight_identifier
1,BHL741,2018-06-17 13:10:00,2018-06-17 13:24:29,ZZZZ,ZZZZ,30,30,60.883333,60.533333,3.500000,2.716667,S92,31.140762,219700570
2,THY3EW,2018-06-17 01:23:00,2018-06-17 01:23:10,UUWW,LTAI,7,10,55.599167,55.590833,37.273056,37.266944,A332,0.541224,219683605
3,THY3EW,2018-06-17 01:23:10,2018-06-17 01:23:56,UUWW,LTAI,10,25,55.590833,55.565833,37.266944,37.249167,A332,1.616653,219683605
4,THY3EW,2018-06-17 01:23:56,2018-06-17 01:25:43,UUWW,LTAI,25,70,55.565833,55.466111,37.249167,37.177500,A332,6.459670,219683605
5,THY3EW,2018-06-17 01:25:43,2018-06-17 01:26:29,UUWW,LTAI,70,90,55.466111,55.416111,37.177500,37.141667,A332,3.238427,219683605


In [7]:
groups = traffic_df.groupby('flight_identifier') # callsign is not unique
print("{} flights in so6 traffic file".format(len(groups)))

34633 flights in so6 traffic file


#### <br> Identify direct flights (if any)

In [8]:
groups.first().query('flight_origin==@from_airport and flight_destination==@to_airport')

,callsign,datetime_begin,datetime_end,flight_origin,flight_destination,fl_begin,fl_end,lat_begin,lat_end,lon_begin,lon_end,aircraft_type,length
flight_identifier,,,,,,,,,,,,,


#### <br> Flights with one connection

In [9]:
first_df = groups.last().query('flight_origin=="LFBO"')
destination_airports = first_df['flight_destination'].unique()
df_list = []
for airport in destination_airports:
    df_list.append(groups.first().query('flight_origin==@airport and flight_destination==@to_airport'))
second_df = pd.concat(df_list)

In [10]:
print('Connecting airports:\n')
connecting_airports = set()
for _, row1 in first_df.iterrows():
    for _, row2 in second_df.iterrows():
        if row1['flight_destination'] == row2['flight_origin']:
            if row2['datetime_begin'] > row1['datetime_end'] + timedelta(hours=min_connecting_time):
                if (row2['datetime_begin'] - row1['datetime_end']).total_seconds() < max_connecting_time * 3600:
                    airline1, airline2 = (row1['callsign'][0:3], row2['callsign'][0:3])
                    if airline1 == airline2:
                        print('-> ' + get_city(row1['flight_destination']), "(", row2['datetime_begin'] - row1['datetime_end'], ")",
                          " with airline:", get_airline(row1['callsign'][0:3]) + '\n')
                        connecting_airports.add(row1['flight_destination'])

Connecting airports:

-> Berlin-Berlin-Schönefeld Airport ( 0 days 01:28:28 )  with airline: easyJet

-> London-London Heathrow Airport ( 0 days 01:27:30 )  with airline: British Airways

-> Munich-Munich Airport ( 0 days 01:28:48 )  with airline: Lufthansa

-> Paris-Charles de Gaulle International Airport ( 0 days 01:12:05 )  with airline: Air France

-> Amsterdam-Amsterdam Airport Schiphol ( 0 days 01:40:43 )  with airline: KLM Royal Dutch Airlines

-> London-London Gatwick Airport ( 0 days 01:34:49 )  with airline: easyJet

-> London-London Heathrow Airport ( 0 days 01:26:02 )  with airline: British Airways

-> Frankfurt-Frankfurt am Main Airport ( 0 days 01:52:17 )  with airline: Lufthansa



In [12]:
connecting_cities = [get_city(ap) for ap in connecting_airports]
print('{} connecting cities found: '.format(len(connecting_cities)))
print(list(connecting_cities))

7 connecting cities found: 
['Munich-Munich Airport', 'London-London Heathrow Airport', 'London-London Gatwick Airport', 'Amsterdam-Amsterdam Airport Schiphol', 'Paris-Charles de Gaulle International Airport', 'Frankfurt-Frankfurt am Main Airport', 'Berlin-Berlin-Schönefeld Airport']
